In [1]:
import pandas as pd
import zipfile
import os

# Path to dataset folder
dataset_path = '/kaggle/input/quora-question-pairs'

# Extract train.csv.zip
with zipfile.ZipFile(os.path.join(dataset_path, 'train.csv.zip'), 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working')

# Extract sample_submission.csv.zip if needed
with zipfile.ZipFile(os.path.join(dataset_path, 'sample_submission.csv.zip'), 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working')

# Load datasets
train_df = pd.read_csv('/kaggle/working/train.csv')
test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))

print(f"Train dataset shape: {train_df.shape}")
print(f"Test dataset shape: {test_df.shape}")


Train dataset shape: (404290, 6)
Test dataset shape: (2345796, 3)


In [3]:
# Install necessary libraries
!pip install sentence-transformers transformers evaluate scikit-learn

# Imports
import zipfile
import os
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from sentence_transformers import CrossEncoder
from torch.utils.data import DataLoader
import random



In [2]:
# 📦 Install dependencies
!pip install -q sentence-transformers scikit-learn

# 📚 Imports
import os, zipfile, random
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.8 MB/s eta 0:00:00:00:0100:01


2025-09-07 11:29:47.661862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757244587.859267      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757244587.916981      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# 🚫 Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"

# 📁 Paths
dataset_path = '/kaggle/input/quora-question-pairs'
working_dir = '/kaggle/working'

# 📂 Unzip train.csv.zip
with zipfile.ZipFile(os.path.join(dataset_path, 'train.csv.zip'), 'r') as zip_ref:
    zip_ref.extractall(working_dir)

# 🧾 Load and sample dataset
train_df = pd.read_csv(os.path.join(working_dir, 'train.csv'))
train_df = train_df.dropna(subset=['question1', 'question2', 'is_duplicate'])
train_df = train_df.sample(frac=0.01, random_state=42)  # ⏱️ small subset for speed

# 🔀 Train/Validation split
train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['is_duplicate'])

# 🧪 Prepare training examples
train_examples = [
    InputExample(texts=[row['question1'], row['question2']], label=float(row['is_duplicate']))
    for _, row in train_df.iterrows()
]

valid_questions1 = valid_df['question1'].tolist()
valid_questions2 = valid_df['question2'].tolist()
valid_labels = valid_df['is_duplicate'].tolist()

# ⚙️ Model setup
model_name = 'sentence-transformers/all-MiniLM-L6-v2'  # 🔧 faster model for quick training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# 🧠 Evaluation function
def evaluate_model(model, questions1, questions2, labels):
    model.eval()
    embeddings1 = model.encode(questions1, convert_to_tensor=True)
    embeddings2 = model.encode(questions2, convert_to_tensor=True)
    cosine_scores = torch.nn.functional.cosine_similarity(embeddings1, embeddings2)
    preds = (cosine_scores > 0.5).long().cpu().numpy()
    score = f1_score(labels, preds)
    return score

# 🚀 Train with CosineSimilarityLoss
print("\nTraining with CosineSimilarityLoss...")
model = SentenceTransformer(model_name, device=device)
loss_fn = losses.CosineSimilarityLoss(model)

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)  # 🔧 smaller batch for speed
print("Starting model.fit()...")
model.fit(
    train_objectives=[(train_dataloader, loss_fn)],
    epochs=1,
    warmup_steps=0,
    show_progress_bar=False  # 🔧 disable progress bar to avoid notebook hang
)
print("Finished training.")

# 📊 Evaluate
f1 = evaluate_model(model, valid_questions1, valid_questions2, valid_labels)
print(f"\n✅ Cosine Similarity Loss F1 Score: {f1:.4f}")

Using device: cuda

Training with CosineSimilarityLoss...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

{'train_runtime': 14.809, 'train_samples_per_second': 218.38, 'train_steps_per_second': 27.348, 'train_loss': 0.17391538266782408, 'epoch': 1.0}
Finished training.


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

Batches:   0%|          | 0/26 [00:00<?, ?it/s]


✅ Cosine Similarity Loss F1 Score: 0.7374
